In [ ]:
import os
from typing import TypedDict, List, Union

from langchain_core.messages import HumanMessage, AIMessage
from langchain_groq import ChatGroq

from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
class AgentState(TypedDict):
    msg: List[Union[HumanMessage, AIMessage]] # -> the message can be of type HumanMessage or AIMessage
    

In [7]:
llm = ChatGroq(model="openai/gpt-oss-120b")   

In [12]:
def process(state:AgentState) -> AgentState:
    response = llm.invoke(state["msg"])
    
    state["msg"].append( AIMessage(content=response.content) )
    
    print(f"\nAI: {response.content}")
    print("CURRENT STATE: ", state["msg"])
    return state

In [13]:
graph = StateGraph(AgentState)
graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END) 
agent = graph.compile()

In [14]:
conv_history = [] # -> NEW!!!

In [16]:
user_input = input("enter: ")

while user_input != "exit":
    # add user input to history
    conv_history.append( HumanMessage(content=user_input) )
    
    result = agent.invoke({
                    "msg": conv_history # -> send the whole history to the llm NOT JUST THE LAST INPUT
                })
    
    print(result['msg'])
    conv_history = result['msg']
    
    user_input = input("enter: ")


AI: Got it! Everything seems to be working fine. Let me know if there's anything you'd like to try or discuss.
CURRENT STATE:  [HumanMessage(content='test', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! It looks like your message came through. How can I assist you today?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='test', additional_kwargs={}, response_metadata={}), AIMessage(content="Got it! Everything seems to be working fine. Let me know if there's anything you'd like to try or discuss.", additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]
[HumanMessage(content='test', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! It looks like your message came through. How can I assist you today?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='test', additional_kwargs={}, response_metadata={}), AIM